In [105]:
import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split

In [106]:
df = pd.read_csv('banana_dataset.csv')
print(df.head())

                  created_at  entry_id  Temperature  Humidity  Ethanol (Gas)  \
0  2022-06-11T15:53:46+00:00         1         34.2      81.6          425.0   
1  2022-06-11T15:54:39+00:00         2         34.3      83.8          425.0   
2  2022-06-11T15:55:22+00:00         3         34.4      84.6          429.0   
3  2022-06-11T15:55:55+00:00         4         34.4      85.7          438.0   
4  2022-06-11T15:56:28+00:00         5         34.5      85.6          424.0   

  Type of Fruit  
0        Banana  
1        Banana  
2        Banana  
3        Banana  
4        Banana  


In [107]:
def synthesize_ethylene(gas):
    ethylene_baseline = 50
    ethylene_increase_rate = 0.5
    peak_gas = 350
    decline_rate = 0.1
    
    if gas < peak_gas:
        return ethylene_baseline + (gas * ethylene_increase_rate)
    else:
        return ethylene_baseline + (peak_gas * ethylene_increase_rate) - ((gas - peak_gas) * decline_rate)


In [108]:
df['Ethylene'] = df['Ethanol (Gas)'].apply(synthesize_ethylene)
df

,created_at,entry_id,Temperature,Humidity,Ethanol (Gas),Type of Fruit,Ethylene
0,2022-06-11T15:53:46+00:00,1,34.2,81.6,425.0,Banana,217.5
1,2022-06-11T15:54:39+00:00,2,34.3,83.8,425.0,Banana,217.5
2,2022-06-11T15:55:22+00:00,3,34.4,84.6,429.0,Banana,217.1
3,2022-06-11T15:55:55+00:00,4,34.4,85.7,438.0,Banana,216.2
4,2022-06-11T15:56:28+00:00,5,34.5,85.6,424.0,Banana,217.6
...,...,...,...,...,...,...,...
2240,2022-06-29T14:27:19+00:00,2241,34.9,96.4,290.0,Banana,195.0
2241,2022-06-29T14:27:53+00:00,2242,35.1,95.8,292.0,Banana,196.0
2242,2022-06-29T14:28:26+00:00,2243,35.2,95.3,299.0,Banana,199.5
2243,2022-06-29T14:29:00+00:00,2244,35.5,94.8,312.0,Banana,206.0


In [109]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2245 entries, 0 to 2244
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   created_at     2245 non-null   object 
 1   entry_id       2245 non-null   int64  
 2   Temperature    2245 non-null   float64
 3   Humidity       2245 non-null   float64
 4   Ethanol (Gas)  2231 non-null   float64
 5   Type of Fruit  2245 non-null   object 
 6   Ethylene       2231 non-null   float64
dtypes: float64(4), int64(1), object(2)
memory usage: 122.9+ KB


In [110]:
df["Grade"] = [0 for i in range (2245)]

# Grade 1: Unripe
df.loc[(df['Ethylene'] < 130) & (df['Ethanol (Gas)'] < 130), 'Grade'] = 1

# Grade 2: Ripe (perfect to eat)
df.loc[((df['Ethylene'] >= 130) & (df['Ethylene'] < 180)) & ((df['Ethanol (Gas)'] >= 130) & (df['Ethanol (Gas)'] < 210)), 'Grade'] = 2

# Grade 3: Over ripe
df.loc[((df['Ethylene'] >= 180) & (df['Ethylene'] < 200)) & ((df['Ethanol (Gas)'] >= 210) & (df['Ethanol (Gas)'] < 360)), 'Grade'] = 3

# Grade 4: Rotting stage
df.loc[(df['Ethylene'] >= 200) | (df['Ethanol (Gas)'] >= 360), 'Grade'] = 4

In [111]:
df_drop = df.drop(['created_at','entry_id' ], axis=1)

In [112]:
df_new = df_drop.dropna()

In [113]:
df_new.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2231 entries, 0 to 2244
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Temperature    2231 non-null   float64
 1   Humidity       2231 non-null   float64
 2   Ethanol (Gas)  2231 non-null   float64
 3   Type of Fruit  2231 non-null   object 
 4   Ethylene       2231 non-null   float64
 5   Grade          2231 non-null   int64  
dtypes: float64(4), int64(1), object(1)
memory usage: 122.0+ KB


In [114]:
X = df_new[['Ethanol (Gas)', 'Ethylene']]
y = df_new['Grade'].values

In [115]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)
model = tf.keras.Sequential([
    tf.keras.layers.Dense(1, activation="sigmoid", input_shape=(5,))
])

c:\Users\USER\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [116]:
model.compile(optimizer='rmsprop', loss="binary_crossentropy", metrics=['accuracy'])

In [117]:
model.fit(X_train, y_train, epochs=50, batch_size=10, verbose=1)

loss, accuracy = model.evaluate(X_test, y_test)
print("Accuracy:", accuracy)

Epoch 1/50


ValueError: Exception encountered when calling Sequential.call().

[1mInput 0 of layer "dense_3" is incompatible with the layer: expected axis -1 of input shape to have value 6, but received input with shape (None, 2)[0m

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=(None, 2), dtype=float32)
  • training=True
  • mask=None